In [1]:
from azure.quantum.optimization import Term
from math import floor, log2
from azure.quantum import Workspace

workspace = Workspace (
  subscription_id = "9d480707-e623-4767-8ecd-61db6aec76b9",
  resource_group = "QAI-1",
  name = "QIO-QAI",
  location = "East US"
)

Your subscription id - 9d480707-e623-4767-8ecd-61db6aec76b9 
And workspace name - QIO-siddhi
Region - East US
Resource group name - QAI-1

In [2]:
def knapsackHamiltonian(costsArray, weightsArray, W):
    
    terms = []

    maxCosts = max(costsArray)

    n = len(costsArray)

    M = floor(log2(W))

    k = [2**i for i in range(M)]
    
    # the mentioned last step
    k.append(W + 1 - 2**M)
    
    # x-Term
    for i in range(n):
        terms.append(Term(c=float(maxCosts * (weightsArray[i]**2) - costsArray[i]), indices=[i]))

    # x-x Term
    for i in range(n):
        for j in range(i+1, n):
            terms.append(Term(c=float(2*maxCosts*weightsArray[i]*weightsArray[j]), indices=[i,j]))

    # x-y Term
    for i in range(n):
        for j in range(M+1):
            terms.append(Term(c=float(-2*maxCosts*weightsArray[i]*k[j]), indices=[i,(n-1)+j]))

    # y Term
    for i in range(M+1):
        terms.append(Term(c=float(maxCosts*(k[i]**2)), indices=[(n-1)+i]))
        
     # y-y Term
    for i in range(M+1):
        for j in range(i+1, M+1):
            terms.append(Term(c=float(2*maxCosts*k[i]*k[j]), indices=[(n-1)+i,(n-1)+j]))

    return terms

In [3]:
from azure.quantum.optimization import Problem, ProblemType
from azure.quantum.optimization import SimulatedAnnealing

In [4]:
costsArray = [5, 4, 2, 1, 4]
weightsArray = [7, 2, 5, 6, 8]
Weight = 15

In [5]:
terms = knapsackHamiltonian(costsArray, weightsArray, Weight)
problem = Problem(name="knapsack problem", problem_type=ProblemType.pubo, terms=terms)
solver = SimulatedAnnealing(workspace, timeout=100, seed=22)

In [6]:
job = solver.submit(problem)
job.refresh()
result = job.get_results()
config = result['configuration']
resultitems = []
for i in config.keys():
    if config[i]:
         resultitems.append(int(i))
    
for i in resultitems:
    if i < len(costsArray):
        print("Picked item number " + str(i))

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code AGV8RBDVC to authenticate.
...Picked item number 0
Picked item number 1
Picked item number 2
